In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import cv2

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
IMG_SIZE = (200,200)
COLOR_MODE = "rgb"
BATCH_SIZE = 64
IMG_SHAPE = IMG_SIZE + (3,) if COLOR_MODE == "rgb" else IMG_SIZE + (1,)

gender_data = './data/gender'

datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.15)

train_gen = datagen.flow_from_directory(
    directory = gender_data,
    subset = 'training',
    target_size= IMG_SIZE,
    color_mode=COLOR_MODE,
    batch_size= BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    directory = gender_data,
    subset = 'validation',
    target_size= IMG_SIZE,
    color_mode=COLOR_MODE,
    batch_size= BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)


Found 20153 images belonging to 2 classes.
Found 3555 images belonging to 2 classes.


In [4]:
# # transfer learning (needs rbg channels)
# preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

# preprocess_input = tf.keras.applications.inception_v3.preprocess_input

# base_model = tf.keras.applications.inception_v3.InceptionV3(
#                 include_top=False, weights='imagenet', 
#                 input_shape=IMG_SHAPE)

# preprocess_input = tf.keras.applications.vgg19.preprocess_input

# base_model = tf.keras.applications.vgg19.VGG19(
#                 include_top=False, weights='imagenet', 
#                 input_shape=IMG_SHAPE)

# preprocess_input = tf.keras.applications.resnet.preprocess_input

# base_model = tf.keras.applications.resnet.ResNet101(
#     include_top=False, weights='imagenet', input_shape=IMG_SHAPE
# )

                                               

# base_model.trainable = False

In [5]:
# inputs = tf.keras.Input(shape=IMG_SHAPE)
# # x = data_augmentation(inputs)
# x = preprocess_input(inputs)
# x = base_model(x, training=False)
# x = layers.Flatten()(x)
# x = layers.Dense(64, activation = 'relu')(x)
# # x = layers.Dense(32, activation = 'relu')(x)
# x = layers.Dense(32, activation = 'relu')(x)
# outputs = layers.Dense(1, activation="sigmoid")(x)
# model = tf.keras.Model(inputs, outputs)


In [6]:
# ~77% val
# model = keras.models.Sequential([
#     layers.Conv2D(32, (2, 2), padding = "same", activation='relu', input_shape=IMG_SHAPE),
#     keras.layers.Dropout(0.2), 
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (2, 2), padding = "same", activation='relu'),
#     keras.layers.Dropout(0.2), 
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (2,2), 2, activation='relu'),
#     keras.layers.Dropout(0.2), 
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (3, 3), 2, activation='relu'),
#     # layers.MaxPooling2D((2, 2)),
#     keras.layers.Flatten(),
#     # keras.layers.Dense(256, activation="relu"),
#     keras.layers.Dense(128, activation="relu"),
#     keras.layers.Dropout(0.2), 
#     # keras.layers.Dense(128, activation="relu"),
#     # keras.layers.Dense(64, activation="relu"),
#     keras.layers.Dense(64, activation="relu"),
#     keras.layers.Dropout(0.2),    
#     keras.layers.Dense(1, activation="sigmoid")
# ])

# model.compile(loss='binary_crossentropy', 
#                 optimizer="Adam", 
#                 metrics=['binary_accuracy'])

In [7]:
# # ~77
# model = keras.models.Sequential([
#     layers.Conv2D(32, (2, 2), padding = "same", activation='relu', input_shape=IMG_SHAPE),
#     keras.layers.Dropout(0.2), 
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (2, 2), padding = "same", activation='relu'),
#     keras.layers.Dropout(0.2), 
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (2,2), activation='relu'),
#     keras.layers.Dropout(0.2), 
#     layers.Conv2D(128, (2,2), 2, activation='relu'),
#     keras.layers.Dropout(0.2), 
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (3, 3), 2, activation='relu'),
#     keras.layers.Flatten(),
#     keras.layers.Dense(128, activation="relu"),
#     keras.layers.Dropout(0.2), 
#     keras.layers.Dense(64, activation="relu"),
#     keras.layers.Dropout(0.2),    
#     keras.layers.Dense(1, activation="sigmoid")
# ])

# model.compile(loss='binary_crossentropy', 
#                 optimizer="Adam", 
#                 metrics=['binary_accuracy'])

In [25]:
def id_block(x, n_filter, filter_size):
    skip = x
    # layer 1
    x = layers.Conv2D(n_filter, filter_size, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # layer 2
    x = layers.Conv2D(n_filter, filter_size, padding = 'same')(x)
    x = layers.BatchNormalization(axis=3)(x)
    x = layers.Activation('relu')(x)
    # skip connection 
    x = tf.keras.layers.Add()([x, skip]) 
    x = layers.Activation('relu')(x)
    return x

In [82]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = layers.Conv2D(64, 2, 2, activation='relu')(inputs)
# x = id_block(x, 64, 3)
# x = layers.MaxPooling2D((2, 2))(x)
x = id_block(x, 64, 3)
# x = layers.MaxPooling2D((3, 3))(x)
x = layers.Conv2D(128, 3, 3, activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.BatchNormalization(axis=3)(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation = 'relu')(x)
# x = layers.Dense(16, activation = 'relu')(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d_135 (Conv2D)             (None, 100, 100, 64) 832         input_34[0][0]                   
__________________________________________________________________________________________________
conv2d_136 (Conv2D)             (None, 100, 100, 64) 36928       conv2d_135[0][0]                 
__________________________________________________________________________________________________
batch_normalization_60 (BatchNo (None, 100, 100, 64) 256         conv2d_136[0][0]                 
___________________________________________________________________________________________

In [83]:
LEARNING_RATE = 0.0005
EPSILON = 0.05

model.compile(loss='binary_crossentropy', 
                optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon = EPSILON), 
                metrics=['accuracy'])

In [84]:
step_train = train_gen.n//train_gen.batch_size
step_val = val_gen.n//val_gen.batch_size
# class_weight = {0:1, 1:10}

model.fit(train_gen, epochs=15, verbose = 1,
         validation_data = val_gen, steps_per_epoch = step_train, validation_steps = step_val)

Epoch 1/15
314/314 [==============================] - 22s 68ms/step - loss: 0.3565 - accuracy: 0.8413 - val_loss: 0.6369 - val_accuracy: 0.6048
Epoch 2/15
314/314 [==============================] - 21s 68ms/step - loss: 0.2308 - accuracy: 0.9071 - val_loss: 0.5054 - val_accuracy: 0.7460
Epoch 3/15
314/314 [==============================] - 21s 68ms/step - loss: 0.1880 - accuracy: 0.9247 - val_loss: 0.5058 - val_accuracy: 0.7548
Epoch 4/15
314/314 [==============================] - 21s 68ms/step - loss: 0.1563 - accuracy: 0.9401 - val_loss: 0.5006 - val_accuracy: 0.7625
Epoch 5/15
314/314 [==============================] - 21s 68ms/step - loss: 0.1291 - accuracy: 0.9547 - val_loss: 0.6236 - val_accuracy: 0.7176
Epoch 6/15
314/314 [==============================] - 22s 68ms/step - loss: 0.1080 - accuracy: 0.9640 - val_loss: 0.6169 - val_accuracy: 0.7469
Epoch 7/15
314/314 [==============================] - 22s 68ms/step - loss: 0.0873 - accuracy: 0.9731 - val_loss: 0.6044 - val_accuracy:

In [87]:
model.save('gender_model.hdf5')